## **Recommender System - for mechanical shops**



In [17]:
import pandas as pd
import numpy as np

In [18]:
Rating_Matrix=pd.read_csv('/content/RatingList - RatingList.csv',encoding ='latin')

In [19]:
Rating_Matrix.head(5)

,userId,mechanical-shop,rating
0,755,Universal service,1
1,755,Rafi mechanical works,5
2,755,VGR,2
3,755,SRV motors,4
4,755,Babu mechanic shop,4


# Read the rating matrix as edge list

In [20]:
Ratings=pd.read_csv('/content/RatingList - RatingList.csv',encoding ='latin')
print('Shape=',Ratings.shape)
print('Rows=',Ratings.shape[0])
print('Columns=',Ratings.shape[1])
print('\n')
print(Ratings.head(5))

Shape= (49, 3)
Rows= 49
Columns= 3


   userId        mechanical-shop  rating
0     755     Universal service        1
1     755  Rafi mechanical works       5
2     755                   VGR        2
3     755             SRV motors       4
4     755     Babu mechanic shop       4


In [21]:
Ratings.head

<bound method NDFrame.head of     userId            mechanical-shop  rating
0      755         Universal service        1
1      755      Rafi mechanical works       5
2      755                       VGR        2
3      755                 SRV motors       4
4      755         Babu mechanic shop       4
5      755                     vijaya       2
6      755                  Rayan pvt       2
7      755         Stand alone motors       3
8      755            Giri automobile       2
9      755                    Mahavir       5
10     755                   Sprinter       2
11     755                     Matrix       4
12     755                MBR motors        2
13     755     ND vehicle care center       5
14    5277         Universal service        5
15    5277      Rafi mechanical works       3
16    5277            Eliyas Mechanic       2
17    5277                 SRV motors       4
18    5277         Babu mechanic shop       2
19    5277                     vijaya       1
20  

# Recommendations based on simple summary statistics
## Task 1: Popularity based on count of ratings

In [22]:
Ratings_count=Ratings.groupby(['mechanical-shop'])[['rating']].count().rename(columns = {'rating': 'Rating_count'}).reset_index()
#print(Ratings_count)
Ratings_count = Ratings_count.sort_values(['Rating_count'], ascending=False)
print(Ratings_count.head(5))

           mechanical-shop  Rating_count
11               Rayan pvt             4
9   ND vehicle care center             3
1       Babu mechanic shop             3
15              Total care             3
14      Stand alone motors             3


## Task 2: Popularity based Mean of ratings
## Calculating mean ratings



In [23]:
Ratings_mean=Ratings.groupby(['mechanical-shop'])[['rating']].mean().rename(columns = {'rating': 'Mean_rating'}).reset_index()
#print(Ratings_mean)
Ratings_mean = Ratings_mean.sort_values(['Mean_rating'], ascending=False)
#print(Ratings_mean)
print(Ratings_mean.head(5))

#Ratings_mean[Ratings_mean['movieId']==1203]

          mechanical-shop  Mean_rating
10  Rafi mechanical works     3.666667
7                 Mahavir     3.666667
2         Eliyas Mechanic     3.500000
12             SRV motors     3.333333
0        Babu automobiles     3.000000


## Task 3: Popularity based on simple summary statistics: Count of ratings above a threshold


In [24]:
Ratings_above_threshold=(Ratings.query('rating>=4')).filter(['mechanical-shop','rating'])
#print(Ratings_above_threshold)

Ratings_count_above_threshold=Ratings_above_threshold.groupby(['mechanical-shop']).count().rename(columns = {'mechanical-shop':'rating'}).reset_index()
#print(Ratings_count_above_threshold)

Ratings_count_above_threshold=Ratings_count_above_threshold.sort_values(['rating'], ascending=False)
print(Ratings_count_above_threshold.head(6))

          mechanical-shop  rating
4                 Mahavir       2
9              SRV motors       2
0      Babu mechanic shop       1
1         Eliyas Mechanic       1
2  Janata vehicles center       1
3             MBR motors        1


## Task 4: Popularity based on Damped mean of ratings
### Damped mean of item $i$ is defined as follows:
# $ damped\_mean(i)= \frac{\sum\limits_{u \in U} {r_{ui} + k\mu}}{n+k}$ 
### Calculating damped mean using k = 5

In [26]:
k=5
Ratings_sum=Ratings.groupby(['mechanical-shop'])[['rating']].sum().rename(columns = {'rating': 'sum_rating'}).reset_index()
#print(Ratings_sum)
#print(Ratings['rating'].mean())

Ratings_sum['sum_rating_factor']=Ratings_sum['sum_rating']+k*(Ratings['rating'].mean())
#print(Ratings_sum)

Ratings_count=Ratings.groupby(['mechanical-shop'])[['rating']].count().rename(columns = {'rating': 'count_rating'}).reset_index()
#print(Ratings_count)

Ratings_count['count_rating_factor']=Ratings_count['count_rating']+k
#print(Ratings_count)

Ratings_damped=pd.merge(Ratings_sum,Ratings_count[['mechanical-shop','count_rating','count_rating_factor']],on=['mechanical-shop'],how='left')
#print(Ratings_damped)

Ratings_damped['damped_mean']=Ratings_damped['sum_rating_factor']/Ratings_damped['count_rating_factor']
#print(Ratings_damped)

Ratings_mean_dampmean=pd.merge(Ratings_mean[['mechanical-shop','Mean_rating']],Ratings_damped[['mechanical-shop','damped_mean']],on=['mechanical-shop'],how='left')
#print(Ratings_mean_dampmean)

# Sorting to get top rated movies
Ratings_mean_dampmean = Ratings_mean_dampmean.sort_values(['damped_mean'], ascending=False)
#print(Ratings_mean_dampmean)

Ratings_mean_dampmean.head(5)

,mechanical-shop,Mean_rating,damped_mean
0,Rafi mechanical works,3.666667,3.109694
1,Mahavir,3.666667,3.109694
3,SRV motors,3.333333,2.984694
2,Eliyas Mechanic,3.500000,2.982507
8,ND vehicle care center,3.000000,2.859694
